In [59]:
import qiskit
from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy
import numpy as np
import random
from random import sample

import math
from qiskit import *
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from math import sqrt, pi
shots = 256

In [60]:
from numpy.random import randint
qn = 100
np.random.seed(seed=7777777)
alice_bits = randint(2, size=qn)
print(alice_bits)
print("Alice's first bit = %i" % alice_bits[0])

[1 0 0 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 1
 1 0 1 1 1 1 0 1 1 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0
 1 0 0 1 0 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 1 0]
Alice's first bit = 1


In [61]:
def encode_message(bits, basis):
    message = [QuantumCircuit(1,1) for i in range(qn)]
    for i in range(qn):
        if basis[i]:
            if bits[i]:    
                message[i].x(0)
            else:
                message[i].h(0)            
        else:
            if bits[i]:    
                message[i].x(0)
    return message

alice_bases = randint(2, size=qn) 
message = encode_message(alice_bits, alice_bases) 
print(alice_bases)
print("Alice's first chosen basis = %i" % alice_bases[0])

[0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 1 0 1 1 1 1 0 1 0 0 1 0 0 0
 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0
 1 1 0 1 1 1 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 1 1 1]
Alice's first chosen basis = 0


In [62]:
def intercept_message(message, basis): 
    backend = Aer.get_backend('qasm_simulator') 
    measurements = [] 
    for q in range(qn):      
        if basis[q]:
            message[q].h(0) 
            message[q].measure(0, 0) 
        else:
            message[q].measure(0, 0) 
        message[q].h(0)
        result = execute(message[q], backend, shots=1, memory=True).result() 
        measured_bit = int(result.get_memory()[0]) 
        measurements.append(measured_bit)
    return measurements


eve_bases = randint(2, size=qn)
intercepted_message = intercept_message(message, eve_bases)
print(eve_bases)
print("Eve's first chosen basis = %i" % alice_bases[0])
print(intercepted_message)
print("Eve's first intercepted message = %i" % intercepted_message[0])

[1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1
 1 0 0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 1 0 1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 1 1 1 1 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 0 0 1 1 0]
Eve's first chosen basis = 0
[1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
Eve's first intercepted message = 1


In [63]:
message = encode_message(intercepted_message, eve_bases)

In [64]:
def measure_message(message, basis):
    backend = Aer.get_backend('qasm_simulator') 
    measurements = [] 
    for q in range(qn): 
        if basis[q]:
            message[q].h(0) 
            message[q].measure(0, 0) 
        else:
            message[q].measure(0, 0) 
        message[q].h(0)
        result = execute(message[q], backend, shots=1, memory=True).result() 
        measured_bit = int(result.get_memory()[0]) 
        measurements.append(measured_bit)
    return measurements

bob_bases = randint(2, size=qn) 
bob_results = measure_message(message, bob_bases) 
print(bob_results)
print("Bob's first chosen basis = %i" % bob_bases[0])

[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Bob's first chosen basis = 1


In [65]:
def remove_garbage(alice_bases, bob_bases, alice_bits):
    return_key = []
    for i in range(qn):
        if alice_bases[i] == bob_bases[i]:
            return_key.append(alice_bits[i])
    return return_key

alice_key = remove_garbage(alice_bases, bob_bases, alice_bits) 
bob_key = remove_garbage(alice_bases, bob_bases, bob_results) 
print(alice_key)
print(bob_key)

[0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]


In [66]:

def calculate_possibility(alice_key, bob_key):
    sum = 0
    num = sample(range(len(alice_key)), len(alice_key) // 2)
    for i in num:
        sum += 1 if alice_key[i] == bob_key[i] else 0
    return sum / len(alice_key)

p = calculate_possibility(alice_key, bob_key)
print("Probability is " , p*100, "%")

Probability is  40.42553191489361 %


Question 3 - (b)

In [67]:
def intercept_message_Briedbart_basis(message, bases): 
    backend = Aer.get_backend('qasm_simulator') 
    measurements = [] 
    for q in range(qn):     
        if bases[q]: 
            message[q].ry(pi*3/4, 0)
            message[q].measure(0, 0) 
        else: 
            message[q].ry(-pi/4, 0)
            message[q].measure(0, 0) 
        message[q].h(0) 
        result = execute(message[q], backend, shots=1, memory=True).result() 
        measured_bit = int(result.get_memory()[0]) 
        measurements.append(measured_bit)
    return measurements

eve_bases = randint(2, size=qn) 
intercepted_message = intercept_message_Briedbart_basis(message, eve_bases)
print(eve_bases)
print("Eve's first chosen basis = %i" % alice_bases[0]) 
print(intercepted_message)
print("Eve's first intercepted message = %i" % intercepted_message[0])

count = 0
for i in range(qn):
    count += 1 if intercepted_message[i] == alice_bits[i] else 0
        
p = count / len(alice_bits)
print("Probability is", p*100, "%")

[1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 1
 0 0 0 1 1 1 1 0 0 0 1 1 1 1 0 0 0 1 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 1 1
 1 1 1 0 1 1 1 1 0 0 0 0 0 1 1 1 0 1 0 0 1 0 0 1 1 1]
Eve's first chosen basis = 0
[0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1]
Eve's first intercepted message = 0
Probability is 54.0 %
